In [ ]:
# Computation Analysis

from synxflow import IO
import numpy as np
import pandas as pd
import os
from shapely.geometry import Point
import geopandas as gpd
import rasterio
from pyproj import Transformer

DEM = IO.Raster('./flooding_data/output.tif')
rain_mask = IO.Raster("./flooding_data/rain_mask.tif")
rain_source = pd.read_csv("./flooding_data/alternate_rain_source.csv", header=None)
rain_source_np = rain_source.to_numpy()

ngpus = 1
case_folder = os.path.join('./', 'flooding_data') # define a case folder in the current directory
case_input = IO.InputModel(DEM, num_of_sections=ngpus, case_folder=case_folder)
case_input.set_initial_condition('h0', 0.0)
case_input.set_rainfall(rain_mask=rain_mask, rain_source=rain_source_np)
case_input.set_grid_parameter(manning=0.035)
case_input.set_runtime([0, 3600, 60, 60])
case_input.write_input_files()

from synxflow import flood
if case_input.num_of_sections > 1:
    flood.run_mgpus(case_input.get_case_folder())
else:
    flood.run(case_input.get_case_folder())

current_path = os.getcwd()

case_input_copy = case_input
case_input_copy.set_case_folder(current_path)

case_output = IO.OutputModel(input_obj = case_input_copy)

gauges_pos, times, values_gauge = case_output.read_gauges_file(file_tag = 'h')

transformer_utm_to_mercator = Transformer.from_crs("epsg:26916", "epsg:3857", always_xy=True)
transformer_mercator_to_latlon = Transformer.from_crs("epsg:3857", "epsg:4326", always_xy=True)

coordinates_list = []

max_depth = case_output.read_grid_file(file_tag='h_max_3600')

with rasterio.open("./output.tif") as src:

    width = src.width
    height = src.height

    transform = src.transform

    for row in range(height):
        for col in range(width):
            lon_utm, lat_utm = transform * (col, row)

            lon_mercator, lat_mercator = transformer_utm_to_mercator.transform(lon_utm, lat_utm)

            lon_lat, lat_lat = transformer_mercator_to_latlon.transform(lon_mercator, lat_mercator)

            coordinates_list.append(Point(lon_lat, lat_lat))

values = max_depth.array.flatten()  # Flatten raster values
values = values.tolist()

while len(values) < len(coordinates_list):
    values.append(0)

gdf_flood = gpd.GeoDataFrame({'value': values, 'geometry': coordinates_list}, geometry="geometry", crs="4326")

gdf_flood = gdf_flood.dropna(subset=["value"])

return times.tolist(), values_gauge.tolist(), gdf_flood

In [ ]:

# # xv, yv = max_depth.to_points()  # Get coordinate grids

# # max_depth.header
# # {'ncols': 337,
# #  'nrows': 393,
# #  'xllcorner': 428427.7933215814,
# #  'yllcorner': 4602438.36674475,
# #  'cellsize': 26.757362390421342,
# #  'NODATA_value': -9999}

# import geopandas as gpd
# import pandas as pd
# from shapely.geometry import Point
# import numpy as np
# import rasterio
# import os

# # Read the raster
# with rasterio.open("./output/h_max_3600.asc") as src:
#     data = src.read(1)  # Read raster values
#     transform = src.transform  # Get transform

# # Get non-NODATA values
# rows, cols = np.where(data != -9999)  # Find valid pixels
# values = data[rows, cols]  # Get corresponding values

# # Convert row, col to coordinates
# x_coords, y_coords = rasterio.transform.xy(transform, rows, cols)

# # Create a GeoDataFrame
# df = pd.DataFrame({"x": x_coords, "y": y_coords, "value": values})
# gdf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df.x, df.y))

# # print(gdf.head())  # Show some points

# # print(gdf.crs)

# gdf = gdf.set_crs(26916)
# # 32616
# gdf = gdf.to_crs(4326)

# gdf.plot()

# # Save as a GeoJSON or Shapefile
# # gdf.to_file("output.geojson", driver="GeoJSON")



In [ ]:
# import rasterio
# from pyproj import Transformer

# transformer_utm_to_mercator = Transformer.from_crs("epsg:26916", "epsg:3857", always_xy=True)
# transformer_mercator_to_latlon = Transformer.from_crs("epsg:3857", "epsg:4326", always_xy=True)

# coordinates_list = []

# with rasterio.open("./output/h_max_3600.asc") as src:

#     width = src.width
#     height = src.height

#     transform = src.transform

#     for row in range(height):
#         for col in range(width):
#             lon_utm, lat_utm = transform * (col, row)

#             lon_mercator, lat_mercator = transformer_utm_to_mercator.transform(lon_utm, lat_utm)

#             lon_lat, lat_lat = transformer_mercator_to_latlon.transform(lon_mercator, lat_mercator)

#             coordinates_list.append((lon_lat, lat_lat))

# print(coordinates_list[:10])

In [ ]:
import pandas as pd

times = arg[0]
values = arg[1]

data = []

for i in range(len(times)):
    data.append({
        'time': times[i],
        'depth': values[i][0],
        'label': 'downstream'
    })
    data.append({
        'time': times[i],
        'depth': values[i][1],
        'label': 'upstream'
    })

df = pd.DataFrame(data)

return df

In [ ]:
# Computation Analysis (run flood and ouput)

# from synxflow import IO

# ngpus = 1
# case_folder = "./data/flood_case"

# from synxflow import flood
# if ngpus > 1:
#     flood.run_mgpus(case_folder)
# else:
#     flood.run(case_folder)

# case_output = IO.OutputModel(input_obj = case_input)

# gauges_pos, times, values = case_output.read_gauges_file(file_tag = 'h')

# return times, values

In [ ]:
# Vega-Lite (connected to Computation Analysis - line chart with times and values)

{
  "$schema": "https://vega.github.io/schema/vega-lite/v5.json",
  "description": "Depth over time for downstream and upstream",
  "width": 600,
  "height": 400,
  "mark": {
    "type": "line",
    "interpolate": "monotone"
  },
  "encoding": {
    "x": {
      "field": "time",
      "type": "quantitative",
      "title": "Time (s)"
    },
    "y": {
      "field": "depth",
      "type": "quantitative",
      "title": "Depth (m)"
    },
    "color": {
      "field": "label",
      "type": "nominal",
      "title": "Measurement",
      "scale": {
        "domain": ["downstream", "upstream"],
        "range": ["#1f77b4", "#ff7f0e"]
      }
    }
  }
}

In [ ]:
# load box
import utk

uc = utk.OSM.load([41.60883899452448, -87.84803061888353, 41.62549084306544, -87.81950563201464], layers=[{'name':'buildings', 'args': {'sizeCells': -1}}])

# buildings
gdf_buildings = uc.layers['gdf']['sections'][0]
gdf_buildings.metadata = {
 'name': 'buildings'
}

return gdf_buildings

In [ ]:
# load box
import utk

uc = utk.OSM.load([41.57124999999999, -87.85847222222222, 41.665138888888876, -87.75125], layers=['surface'])

#surface
json_surface = uc.layers['json'][0]
gdf_surface = uc.layers['gdf']['objects'][0]
gdf_surface.metadata = {
 'name': 'surface'
}

return gdf_surface

In [ ]:
# Data tranformation (connected to gdf_surface)

import geopandas as gpd
from shapely.geometry import Polygon, MultiPolygon
import numpy as np

gdf_surface = arg

minx, miny, maxx, maxy = gdf_surface.geometry.total_bounds
cell_size = 50

grid_cells = []
for x0 in np.arange(minx, maxx, cell_size):
    for y0 in np.arange(miny, maxy, cell_size):
        # 1st cell boundary
        x1 = x0 + cell_size
        y1 = y0 + cell_size
        grid_cells.append(Polygon([(x0, y0), (x1, y0), (x1, y1), (x0, y1)]))

grid_gdf = gpd.GeoDataFrame(geometry=grid_cells, crs="EPSG:3395")

grid_gdf = grid_gdf.reset_index(drop=True)

grid_gdf.metadata = {
    'name': 'grid'
}

return grid_gdf

In [ ]:
# Merge grid_flood and grid_gdf

In [ ]:
# Computation analysis (color grid with flood values)

import geopandas as gpd

gdf_flood = arg[0]
grid_gdf = arg[1]

gdf_flood = gdf_flood.set_crs(4326)
grid_gdf = grid_gdf.set_crs(3395)

gdf_flood = gdf_flood.to_crs(3395)

joined = gpd.sjoin(grid_gdf, gdf_flood, how='left', predicate='contains')

grid_gdf['average_value'] = joined.groupby(joined.index)['value'].mean()

grid_gdf = grid_gdf[["geometry", "average_value"]]
grid_gdf = grid_gdf.rename(columns={"average_value": "value"})

grid_gdf = grid_gdf.dropna(subset=["value"])

grid_gdf.metadata = {
    'name': 'ground'
}

return grid_gdf

In [ ]:
# Merge (grid_gdf, gdf_buildings)

In [ ]:
# Computation analysis

import geopandas as gpd

grid_gdf = arg[0]
gdf_buildings = arg[1]

grid_gdf = grid_gdf.set_crs(3395)
gdf_buildings = gdf_buildings.set_crs(4326)
gdf_buildings = gdf_buildings.to_crs(3395)

joined = gpd.sjoin(gdf_buildings, grid_gdf, how='left', predicate='intersects')

gdf_buildings['thematic'] = joined.groupby(joined.index)['value'].mean()

gdf_buildings["thematic"] = gdf_buildings["thematic"].fillna(0.1)

gdf_buildings.metadata = {
 'name': 'buildings'
}

return gdf_buildings

In [ ]:
# Merge (grid_gdf, gdf_buildings)

In [ ]:
# UTK

# interpolateViridis
# "color_map": "interpolateBlues"